In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

In [ ]:
# Data Collection (From Yahoo Finance)

# extract all the stock tickers from S&P 500
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers = tickers['Symbol'].tolist()
# Replace . with - to meet Yahoo Finance ticker's format

tickers = [ticker.replace('.', '-') for ticker in tickers]
#print(tickers[:])
#len(tickers) # Ensure the total number of stock = 503


# Extract the S&P 500 stock prices for 2021 through 1May2024. We use Adjusted close price.
ticker_data = yf.download(tickers, start="2021-01-01", end="2024-07-01")
ticker_data.head()

[*********************100%%**********************]  503 of 503 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SW']: YFChartError("%ticker%: Data doesn't exist for startDate = 1609477200, endDate = 1719806400")


Price        Adj Close                                                        \
Ticker               A    AAL        AAPL       ABBV        ABNB         ABT   
Date                                                                           
2021-01-04  115.980728  15.13  126.830070  90.489510  139.149994  102.054939   
2021-01-05  116.928993  15.43  128.398178  91.425224  148.300003  103.317650   
2021-01-06  120.135468  15.52  124.076080  90.635445  142.770004  103.102531   
2021-01-07  123.332176  15.38  128.309998  91.605484  151.270004  104.103325   
2021-01-08  124.211990  15.13  129.417450  92.086227  149.770004  104.393288   

Price                                                      ...   Volume  \
Ticker           ACGL         ACN        ADBE         ADI  ...      WTW   
Date                                                       ...            
2021-01-04  34.900002  243.104004  485.339996  137.128555  ...  1654000   
2021-01-05  35.040001  244.487991  485.690002  139.579605  ...  1435000   
2021-01-06  36.580002  247.161072  466.309998  140.208817  ...  1792300   
2021-01-07  36.240002  249.493011  477.739990  146.134613  ...  1324300   
2021-01-08  36.439999  250.403000  485.100006  147.195755  ...   925300   

Price                                                                       \
Ticker           WY     WYNN      XEL       XOM      XYL      YUM      ZBH   
Date                                                                         
2021-01-04  4859100  3596800  3197100  27764700   814700  2058200  1265870   
2021-01-05  3909000  1814600  1976800  44035100  1005200  1475300   998894   
2021-01-06  5602100  2433700  2667000  36484800  1015300  1352700  1424799   
2021-01-07  3984700  3846700  2630400  29528100  1539300  2222900  1237648   
2021-01-08  4376000  2812400  2221500  28628200  1632200  1252200  1259072   

Price                        
Ticker        ZBRA      ZTS  
Date                         
2021-01-04  432600  1805900  
2021-01-05  374700  1358600  
2021-01-06  419500  2139800  
2021-01-07  379900  1710700  
2021-01-08  348500  1475900  

[5 rows x 3018 columns]

In [ ]:
#get news sentiment data and select correct time period
news_sentiment_data = pd.read_excel('https://www.frbsf.org/wp-content/uploads/news_sentiment_data.xlsx?20240105', sheet_name='Data')
pd.to_datetime(news_sentiment_data['date'])
news_sentiment_data = news_sentiment_data.rename(columns={'date':'Date'})
news_sentiment_data = news_sentiment_data.set_index(['Date'])
news_sentiment_data = news_sentiment_data.loc['2021-01-01':'2024-07-01']
news_sentiment_data.head()

,News Sentiment
Date,
2021-01-01,-0.121698
2021-01-02,-0.103407
2021-01-03,-0.101170
2021-01-04,-0.097181
2021-01-05,-0.102694


In [ ]:
#get daily treasury real long term rates, has to be done by year due to the US Treasury website
daily_treasury_rltr_2021 = pd.read_csv('https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/2021/all?type=daily_treasury_real_long_term&field_tdr_date_value=2021&page&_format=csv')
daily_treasury_rltr_2022 = pd.read_csv('https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/2022/all?type=daily_treasury_real_long_term&field_tdr_date_value=2022&page&_format=csv')
daily_treasury_rltr_2023 = pd.read_csv('https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/2023/all?type=daily_treasury_real_long_term&field_tdr_date_value=2023&page&_format=csv')
daily_treasury_rltr_2024 = pd.read_csv('https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/2024/all?type=daily_treasury_real_long_term&field_tdr_date_value=2024&page&_format=csv')

In [ ]:
#concatenate and reformat the real long term rate data to match other datasets
real_long_term_rates = pd.concat([daily_treasury_rltr_2021, daily_treasury_rltr_2022, daily_treasury_rltr_2023, daily_treasury_rltr_2024])
real_long_term_rates = real_long_term_rates.rename(columns={'LT Real Average (10> Yrs)':'LTRA'})
real_long_term_rates['Date'] = pd.to_datetime(real_long_term_rates['Date'])
real_long_term_rates = real_long_term_rates.set_index(['Date'])
real_long_term_rates = real_long_term_rates.sort_index()
real_long_term_rates = real_long_term_rates.loc['2021-01-01':'2024-07-01']

real_long_term_rates.head()

,LTRA
Date,
2021-01-04,-0.50
2021-01-05,-0.47
2021-01-06,-0.41
2021-01-07,-0.39
2021-01-08,-0.33


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#US GDP data, quarterly values to be forward-filled, csv available at https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?locations=US
GDP_data = pd.read_csv('/content/drive/MyDrive/Wealth_Watchers/GDPPOT.csv')
GDP_data = GDP_data.rename(columns={'DATE':'Date', 'GDPPOT': 'USGDP'})
GDP_data['Date'] = pd.to_datetime(GDP_data['Date'])
GDP_data = GDP_data.set_index(['Date'])
GDP_data = GDP_data.sort_index()
GDP_data = GDP_data.loc['2021-01-01':'2024-07-01']
GDP_data.head()


,USGDP
Date,
2021-01-01,21261.600036
2021-04-01,21355.977383
2021-07-01,21454.647379
2021-10-01,21558.240246
2022-01-01,21667.693841


In [ ]:
#US Inflation data, monthly values to be forward-filled, data available at https://ycharts.com/indicators/us_inflation_rate
USIR_data = pd.read_csv('/content/drive/MyDrive/Wealth_Watchers/USIR.csv')
USIR_data = USIR_data.rename(columns={'Inflation Rate':'USIR'})
USIR_data['Date'] = pd.to_datetime(USIR_data['Date'])
USIR_data = USIR_data.set_index(['Date'])
USIR_data = USIR_data.sort_index()
USIR_data = USIR_data.loc['2020-12-31':'2024-07-01']
USIR_data.head()

,USIR
Date,
2020-12-31,1.36
2021-01-31,1.40
2021-02-28,1.68
2021-03-31,2.62
2021-04-30,4.16


In [ ]:
#US Unemployment Rate data, monthly, to be forward filled, data available at https://data.bls.gov/timeseries/LNS14000000
US_unemployment_data = pd.read_csv('/content/drive/MyDrive/Wealth_Watchers/USUnemploymentRate.csv')
US_unemployment_data['Date'] = pd.to_datetime(US_unemployment_data['Date'])
US_unemployment_data = US_unemployment_data.set_index(['Date'])
US_unemployment_data = US_unemployment_data.sort_index()
US_unemployment_data = US_unemployment_data.loc['2020-12-31':'2024-07-01']
US_unemployment_data.head()

,US Unemployment Rate
Date,
2020-12-31,6.7
2021-01-31,6.4
2021-02-28,6.2
2021-03-31,6.1
2021-04-30,6.1


In [ ]:
#import daily Yuan to USD exchange rate from local file
yuan_usd_daily = pd.read_csv('/content/drive/MyDrive/Wealth_Watchers/Dollar_Yuan_CNY=X.csv')

#reformat to match
yuan_usd_daily['Date'] = pd.to_datetime(yuan_usd_daily['Date'])
yuan_usd_daily = yuan_usd_daily.set_index(['Date'])
yuan_usd_daily = yuan_usd_daily.sort_index()
yuan_usd_daily = yuan_usd_daily.loc['2021-01-01':'2024-07-01']
yuan_usd_daily = yuan_usd_daily.drop(['Volume', 'Open', 'High', 'Low', 'Close'], axis=1)
yuan_usd_daily = yuan_usd_daily.rename(columns={'Adj Close':'Yuan Value in USD'})

yuan_usd_daily.head()

,Yuan Value in USD
Date,
2021-01-01,6.5317
2021-01-04,6.5320
2021-01-05,6.4605
2021-01-06,6.4552
2021-01-07,6.4607


In [ ]:
#these are eight stocks used as placeholders, once stocks have been identified by the unsupervised analysis these will be replaced

stock_prices_daily = ticker_data[[('Adj Close', 'SMCI'), ('Adj Close', 'FSLR'), ('Adj Close', 'NVDA'), ('Adj Close', 'MRNA'),
                           ('Adj Close', 'KO'), ('Adj Close', 'RSG'), ('Adj Close', 'WMT'), ('Adj Close', 'ROP')]]
stock_prices_daily.head()

Price       Adj Close                                                \
Ticker           SMCI        FSLR       NVDA        MRNA         KO   
Date                                                                  
2021-01-04  31.049999  101.239998  13.082322  111.730003  47.408409   
2021-01-05  31.160000   92.160004  13.372880  109.180000  46.887238   
2021-01-06  31.889999   99.690002  12.584509  116.260002  45.395618   
2021-01-07  32.189999  105.500000  13.312274  115.089996  44.892422   
2021-01-08  31.879999  104.099998  13.245185  112.750000  45.898815   

Price                                         
Ticker            RSG        WMT         ROP  
Date                                          
2021-01-04  89.061890  46.305023  410.800537  
2021-01-05  89.157112  46.058529  410.438263  
2021-01-06  91.289879  46.346092  414.570251  
2021-01-07  92.137268  46.342945  422.161896  
2021-01-08  92.327698  46.336624  422.671722

In [ ]:
df_list = [stock_prices_daily, yuan_usd_daily, US_unemployment_data, USIR_data, GDP_data, real_long_term_rates, news_sentiment_data]
all_columns_df = pd.concat(df_list, axis=1)
all_columns_df = all_columns_df.fillna(method='ffill')
all_columns_df = all_columns_df.fillna(method='bfill')
all_columns_df.head(10)

,"(Adj Close, SMCI)","(Adj Close, FSLR)","(Adj Close, NVDA)","(Adj Close, MRNA)","(Adj Close, KO)","(Adj Close, RSG)","(Adj Close, WMT)","(Adj Close, ROP)",Yuan Value in USD,US Unemployment Rate,USIR,USGDP,LTRA,News Sentiment
Date,,,,,,,,,,,,,,
2020-12-31,31.049999,101.239998,13.082322,111.730003,47.408409,89.061890,46.305023,410.800537,6.5317,6.7,1.36,21261.600036,-0.50,-0.121698
2021-01-01,31.049999,101.239998,13.082322,111.730003,47.408409,89.061890,46.305023,410.800537,6.5317,6.7,1.36,21261.600036,-0.50,-0.121698
2021-01-02,31.049999,101.239998,13.082322,111.730003,47.408409,89.061890,46.305023,410.800537,6.5317,6.7,1.36,21261.600036,-0.50,-0.103407
2021-01-03,31.049999,101.239998,13.082322,111.730003,47.408409,89.061890,46.305023,410.800537,6.5317,6.7,1.36,21261.600036,-0.50,-0.101170
2021-01-04,31.049999,101.239998,13.082322,111.730003,47.408409,89.061890,46.305023,410.800537,6.5320,6.7,1.36,21261.600036,-0.50,-0.097181
2021-01-05,31.160000,92.160004,13.372880,109.180000,46.887238,89.157112,46.058529,410.438263,6.4605,6.7,1.36,21261.600036,-0.47,-0.102694
2021-01-06,31.889999,99.690002,12.584509,116.260002,45.395618,91.289879,46.346092,414.570251,6.4552,6.7,1.36,21261.600036,-0.41,-0.096816
2021-01-07,32.189999,105.500000,13.312274,115.089996,44.892422,92.137268,46.342945,422.161896,6.4607,6.7,1.36,21261.600036,-0.39,-0.080024
2021-01-08,31.879999,104.099998,13.245185,112.750000,45.898815,92.327698,46.336624,422.671722,6.4772,6.7,1.36,21261.600036,-0.33,-0.083688
